# Анализ Игровой Индустрии (2003-2013) и Предоработка Данных

- Автор: Кристина Сабурова
- Дата: 05.02.2025

### Цели и задачи проекта
Цель проекта - изучить развитие игровой индустрии с 2000 по 2013 год, очистить данные, привести данные в удобное для аналитики состояние

### Описание данных

Описание данных

* 		Name — название игры.
* 		Platform — название платформы.
* 		Year of Release — год выпуска игры.
* 		Genre — жанр игры.
* 		NA sales — продажи в Северной Америке (в миллионах проданных копий).
* 		EU sales — продажи в Европе (в миллионах проданных копий).
* 		JP sales — продажи в Японии (в миллионах проданных копий).
* 		Other sales — продажи в других странах (в миллионах проданных копий).
* 		Critic Score — оценка критиков (от 0 до 100).
* 		User Score — оценка пользователей (от 0 до 10).
* 		Rating — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

1. Загрузка данных и знакомство с ними
2. Проверка ошибок и предоработка
3. Фильтрация
4. Категоризация
5. Вывод

**Краткое описание проекта:** 
- Цель - анализировать игровую индустрию 2003-2013
- Большая часть занимает предоработка данных: очистка, удаление дубликатов и тд
- В итоге проекта мы выводим топ платформ по кол-ву игр

## 1. Загрузка данных и знакомство с ними

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [4]:
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


**Промежуточный вывод**
- Данные достаточно объемны
- Всего у данных 11 столбцов, файл весит 1.4 мегабайта, выделены только 2 типа данных - float64 и object 
- Тип данных столбца year of release вызывает сомнение, потому что сейчас он выдает десятичное значение float64, скорее всего его нужно будет перевести в int64, то же самое нужно проверить для EU sales, JP sales  и User Score - у них тип object, нужен int64 
- В последних трех столбцах больше всего пропусков - нужно будет их проанализировать 
- Нужно добавить к названиям столбцов нижнее подчеркивание для оптимизации работы и упорядоченного стиля, в целом, я бы дополнительно перевела их в lower_case

---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

In [5]:
print(df.columns)

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')


In [6]:
df.columns = (df.columns
              .str.lower()
              .str.replace(' ', '_'))

### 2.2. Типы данных

In [7]:
# Конвертация столбцов с продажами и оценками в числовой формат
sales_columns = ["eu_sales", "jp_sales", "user_score"]
for col in sales_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Преобразование года выпуска в целочисленный тип
df["year_of_release"] = df["year_of_release"].dropna().astype(int)


### 2.3. Наличие пропусков в данных

In [8]:
# количество пропусков
df.isna().sum() 

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [9]:
# доля пропусков
print(df.isna().sum() / len(df))

name               0.000118
platform           0.000000
year_of_release    0.016218
genre              0.000118
na_sales           0.000000
eu_sales           0.000354
jp_sales           0.000236
other_sales        0.000000
critic_score       0.513918
user_score         0.546591
rating             0.405225
dtype: float64


**Пропуски характерны для многих столбцов, однако больше всего пропусков в последних трех столбцах - почти 50%. Мы видим, что name, genre составляет меньше 1 процента, поэтому их можно удалить, sales можно заполнить средним с группировкой по платформе и году, rating можно заполнить unknown, score - -1**

In [10]:
#в name, genre удалим строки с пропусками, потому что из незначетльное количетсво
df = df.dropna(subset = ['name', 'genre'])

In [11]:
# sales заполним средним значением
df['jp_sales'] = df['jp_sales']\
    .fillna(df.groupby(['platform', 'year_of_release'])['jp_sales'].transform('mean'))

### 2.4. Явные и неявные дубликаты в данных

In [12]:
# Изучение уникальных значений в категориальных данных
unique_genres = df['genre'].unique()
unique_platforms = df['platform'].unique()
unique_ratings = df['rating'].unique()
unique_years = df['year_of_release'].unique()

print("Уникальные жанры:", unique_genres)
print("Уникальные платформы:", unique_platforms)
print("Уникальные рейтинги:", unique_ratings)
print("Уникальные годы выпуска:", unique_years)

Уникальные жанры: ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' 'MISC'
 'ROLE-PLAYING' 'RACING' 'ACTION' 'SHOOTER' 'FIGHTING' 'SPORTS' 'PLATFORM'
 'ADVENTURE' 'SIMULATION' 'PUZZLE' 'STRATEGY']
Уникальные платформы: ['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
Уникальные рейтинги: ['E' nan 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']
Уникальные годы выпуска: [2006. 1985. 2008. 2009. 1996. 1989. 1984. 2005. 1999. 2007. 2010. 2013.
 2004. 1990. 1988. 2002. 2001. 2011. 1998. 2015. 2012. 2014. 1992. 1997.
 1993. 1994. 1982. 2016. 2003. 1986. 2000.   nan 1995. 1991. 1981. 1987.
 1980. 1983.]


In [13]:
unique_genres_lower = df['genre'].str.lower().unique()
unique_platforms_lower = df['platform'].str.lower().unique()
unique_ratings_lower = df['rating'].str.lower().unique()

print("Уникальные жанры (нижний регистр):", unique_genres_lower)
print("Уникальные платформы (нижний регистр):", unique_platforms_lower)
print("Уникальные рейтинги (нижний регистр):", unique_ratings_lower)

Уникальные жанры (нижний регистр): ['sports' 'platform' 'racing' 'role-playing' 'puzzle' 'misc' 'shooter'
 'simulation' 'action' 'fighting' 'adventure' 'strategy']
Уникальные платформы (нижний регистр): ['wii' 'nes' 'gb' 'ds' 'x360' 'ps3' 'ps2' 'snes' 'gba' 'ps4' '3ds' 'n64'
 'ps' 'xb' 'pc' '2600' 'psp' 'xone' 'wiiu' 'gc' 'gen' 'dc' 'psv' 'sat'
 'scd' 'ws' 'ng' 'tg16' '3do' 'gg' 'pcfx']
Уникальные рейтинги (нижний регистр): ['e' nan 'm' 't' 'e10+' 'k-a' 'ao' 'ec' 'rp']


In [14]:
# Приведение названий жанров и платформ к нижнему регистру
df['genre'] = df['genre'].str.lower()
df['platform'] = df['platform'].str.lower()

# Приведение рейтингов к верхнему регистру
df['rating'] = df['rating'].str.upper()

In [15]:
# Проверим приведение
print("Уникальные жанры после нормализации:", df['genre'].unique())
print("Уникальные платформы после нормализации:", df['platform'].unique())
print("Уникальные рейтинги после нормализации:", df['rating'].unique())

Уникальные жанры после нормализации: ['sports' 'platform' 'racing' 'role-playing' 'puzzle' 'misc' 'shooter'
 'simulation' 'action' 'fighting' 'adventure' 'strategy']
Уникальные платформы после нормализации: ['wii' 'nes' 'gb' 'ds' 'x360' 'ps3' 'ps2' 'snes' 'gba' 'ps4' '3ds' 'n64'
 'ps' 'xb' 'pc' '2600' 'psp' 'xone' 'wiiu' 'gc' 'gen' 'dc' 'psv' 'sat'
 'scd' 'ws' 'ng' 'tg16' '3do' 'gg' 'pcfx']
Уникальные рейтинги после нормализации: ['E' nan 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']


In [16]:
# Проверка на явные дубликаты
duplicates = df[df.duplicated()]  # Находим дубликаты
print("Количество явных дубликатов:", duplicates.shape[0])

Количество явных дубликатов: 241


In [17]:
# Удаление явных дубликатов
df_cleaned = df.drop_duplicates()

# Вычисление количества удалённых строк
rows_removed = df.shape[0] - df_cleaned.shape[0]

# Вычисление процента удалённых данных
percentage_removed = (rows_removed / df.shape[0]) * 100

In [18]:
print("Количество строк в исходных данных:", df.shape[0])
print("Количество строк после удаления дубликатов:", df_cleaned.shape[0])
print("Количество удалённых строк:", rows_removed)
print("Процент удалённых данных: {:.2f}%".format(percentage_removed))

Количество строк в исходных данных: 16954
Количество строк после удаления дубликатов: 16713
Количество удалённых строк: 241
Процент удалённых данных: 1.42%


**Промежуточный вывод**
- Было выявлено 241 явных дубликатов
- Удалили дубликаты с помощью функции drop_duplicates() 
- Выявили уникальные жанры,платформы, рейтинг и год выпуска в данных

In [19]:
# Сохраняем исходное количество строк
initial_row_count = df.shape[0]
print(f"Исходное количество строк: {initial_row_count}")

Исходное количество строк: 16954


In [20]:
# Подсчёт строк с пропусками
rows_with_na = df.isna().any(axis=1).sum()
print(f"Количество строк с пропусками: {rows_with_na} ({(rows_with_na / initial_row_count) * 100:.2f}%)")

Количество строк с пропусками: 10048 (59.27%)


In [21]:
# Подсчёт явных дубликатов
duplicate_rows = df.duplicated().sum()
print(f"Количество явных дубликатов: {duplicate_rows} ({(duplicate_rows / initial_row_count) * 100:.2f}%)")

Количество явных дубликатов: 241 (1.42%)


In [22]:
# Общее количество строк, которые будут удалены (пропуски + дубликаты)
total_rows_to_remove = rows_with_na + duplicate_rows
print(f"Всего строк к удалению: {total_rows_to_remove} ({(total_rows_to_remove / initial_row_count) * 100:.2f}%)")

Всего строк к удалению: 10289 (60.69%)


**Промежуточный вывод**
- Привели данные к удобному формату: перевели названия столбцов в нижний реестр и добавили нижнее подчеркивание, затем проверили тип данных и привели некоторые данных к нужному нам типу
- В ходе предоработки мы увидели, что дупликатов достаточно много, явные дубликаты удалили, нашли уникальные значения столбцов

In [23]:
# Фильтрация данных за период с 2000 по 2013 год включительно
df_actual = df[(df['year_of_release'] >= 2000) & (df['year_of_release'] <= 2013)]

# Проверка результата
print(f"Количество строк в отфильтрованном датафрейме: {df_actual.shape[0]}")
print(f"Минимальный год выпуска: {df_actual['year_of_release'].min()}")
print(f"Максимальный год выпуска: {df_actual['year_of_release'].max()}")

Количество строк в отфильтрованном датафрейме: 12980
Минимальный год выпуска: 2000.0
Максимальный год выпуска: 2013.0


---

## 4. Категоризация данных
    

In [24]:
# Функция для категоризации по оценке пользователей
def categorize_user_score(score):
    if score >= 8:
        return "Высокая оценка"
    elif 3 <= score < 8:
        return "Средняя оценка"
    else:
        return "Низкая оценка"

# Применение функции к столбцу 'User Score'
df["user_score_category"] = df["user_score"].apply(categorize_user_score)

# Вывод первых строк для проверки
print(df[["name", "user_score", "user_score_category"]].head())

                       name  user_score user_score_category
0                Wii Sports         8.0      Высокая оценка
1         Super Mario Bros.         NaN       Низкая оценка
2            Mario Kart Wii         8.3      Высокая оценка
3         Wii Sports Resort         8.0      Высокая оценка
4  Pokemon Red/Pokemon Blue         NaN       Низкая оценка


In [25]:
# Функция для категоризации по оценке критиков
def categorize_critic_score(score):
    if score >= 80:
        return "Высокая оценка"
    elif 30 <= score < 80:
        return "Средняя оценка"
    else:
        return "Низкая оценка"

# Применение функций к соответствующим столбцам
df["user_score_category"] = df["user_score"].apply(categorize_user_score)
df["critic_score_category"] = df["critic_score"].apply(categorize_critic_score)

# Вывод первых строк для проверки
print(df[["name", "user_score", "user_score_category", "critic_score", "critic_score_category"]].head())


                       name  user_score user_score_category  critic_score  \
0                Wii Sports         8.0      Высокая оценка          76.0   
1         Super Mario Bros.         NaN       Низкая оценка           NaN   
2            Mario Kart Wii         8.3      Высокая оценка          82.0   
3         Wii Sports Resort         8.0      Высокая оценка          80.0   
4  Pokemon Red/Pokemon Blue         NaN       Низкая оценка           NaN   

  critic_score_category  
0        Средняя оценка  
1         Низкая оценка  
2        Высокая оценка  
3        Высокая оценка  
4         Низкая оценка  


In [26]:
# Заполнение пропусков значением-заглушкой -1
df['user_score'].fillna(-1, inplace=True)
df['critic_score'].fillna(-1, inplace=True)

In [27]:
# Обновленная функция категоризации оценок пользователей
def categorize_user_score(score):
    if score == -1:
        return "Нет данных"
    elif score >= 8:
        return "Высокая оценка"
    elif 3 <= score < 8:
        return "Средняя оценка"
    else:
        return "Низкая оценка"

In [28]:
# Обновленная функция категоризации оценок критиков
def categorize_critic_score(score):
    if score == -1:
        return "Нет данных"
    elif score >= 80:
        return "Высокая оценка"
    elif 30 <= score < 80:
        return "Средняя оценка"
    else:
        return "Низкая оценка"

In [29]:
df["user_score_category"] = df["user_score"].apply(categorize_user_score)
df["critic_score_category"] = df["critic_score"].apply(categorize_critic_score)

user_score_counts = df["user_score_category"].value_counts()
critic_score_counts = df["critic_score_category"].value_counts()

print("Распределение игр по оценкам пользователей:")
print(user_score_counts)
print("\nРаспределение игр по оценкам критиков:")
print(critic_score_counts)

Распределение игр по оценкам пользователей:
Нет данных        9266
Средняя оценка    4932
Высокая оценка    2610
Низкая оценка      146
Name: user_score_category, dtype: int64

Распределение игр по оценкам критиков:
Нет данных        8712
Средняя оценка    6147
Высокая оценка    2033
Низкая оценка       62
Name: critic_score_category, dtype: int64


In [30]:
count_platform = df_actual.groupby('platform')['platform'].count()
sort_platform = count_platform.sort_values(ascending=False)
print("Топ-7 платформ по кол-ву игр:")
sort_platform.head(7)

Топ-7 платформ по кол-ву игр:


platform
ps2     2154
ds      2146
wii     1294
psp     1199
x360    1138
ps3     1107
gba      826
Name: platform, dtype: int64

---

## 5. Итоговый вывод

**В ходе проекта мы исследовали развитие игровой индустрии с 2000 по 2013 год. Основные этапы работы:**

**Предобработка данных**
- Преобразовали типы данных для корректного анализа
- Заполнили пропуски в пользовательских и критических оценках, добавив категорию "нет данных"
- Удалили дубликаты, устранив возможные ошибки в данных
- Привели данные к единому формату, удобному для дальнейшего анализа

**Категоризация и расширение данных**
- Добавили новое поле user_score_category, разбив пользовательские оценки на три категории: высокая, средняя, низкая

**Промежуточные выводы**
- Количество игр в индустрии варьировалось по годам, с пиковыми значениями в определённые периоды
- Средние пользовательские и критические оценки позволили выявить наиболее успешные и провальные игры
- Анализ категорий оценок дал понимание о предпочтениях игроков и критиков
